In [1]:
import os

os.chdir('..')
print(os.getcwd())

/workspace/LLaSA


In [2]:
import importlib

importlib.reload(importlib.import_module("model.llasa_arch"))
#importlib.reload(importlib.import_module("model.llasa_processor"))
#importlib.reload(importlib.import_module("utils.dataloader"))

/opt/conda/envs/llasa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'model.llasa_arch' from '/workspace/LLaSA/model/llasa_arch.py'>

In [2]:
# reload module 

import torch
from transformers import BitsAndBytesConfig, LlavaNextVideoForConditionalGeneration
from model.llasa_arch import LLaSA
from model.llasa_processor import LlavaNextVideoProcessor


quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

#del model

processor = LlavaNextVideoProcessor.from_pretrained("/workspace/LLaSA/checkpoints/LLaVA-NeXT-Video-7B-hf", cache_dir='/workspace/LLaSA/checkpoints')
model = LLaSA.from_pretrained(
    "/workspace/LLaSA/checkpoints/LLaVA-NeXT-Video-7B-hf",
    quantization_config=quantization_config,
    device_map='auto',
    cache_dir = '/workspace/LLaSA/checkpoints'
)


/opt/conda/envs/llasa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Unrecognized keys in `rope_scaling` for 'rope_type'='linear': {'type'}
Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.32s/it]
Some weights of LLaSA were not initialized from the model checkpoint at /workspace/LLaSA/checkpoints/LLaVA-NeXT-Video-7B-hf and are newly initialized: ['seg_projector.linear.bias', 'seg_projector.linear.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [4]:
from utils.dataloader import Dataset_A2D
train_dataset = Dataset_A2D(processor=processor,start_idx=0, end_idx=3500)
eval_dataset = Dataset_A2D(processor=processor,start_idx=3500, end_idx=3600)

In [5]:
from peft import LoraConfig


lora_config = LoraConfig(
    target_modules=["q_proj", "k_proj"],
    init_lora_weights=False
)

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

model.add_adapter(lora_config, adapter_name="Llasa_adapter")
model.set_adapter('Llasa_adapter')

In [6]:

for name, param in model.named_parameters():
    if "seg_projector" in name:
        if not torch.is_floating_point(param):
            param.data = param.data.float()  # Convert to float32 if not already
            # Alternatively, you can use double() or half() for float64 or float16
        param.requires_grad = True
    else:
        continue

for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Layer: {name} | Trainable")


Layer: vision_tower.vision_model.encoder.layers.0.self_attn.k_proj.lora_A.Llasa_adapter.weight | Trainable
Layer: vision_tower.vision_model.encoder.layers.0.self_attn.k_proj.lora_B.Llasa_adapter.weight | Trainable
Layer: vision_tower.vision_model.encoder.layers.0.self_attn.q_proj.lora_A.Llasa_adapter.weight | Trainable
Layer: vision_tower.vision_model.encoder.layers.0.self_attn.q_proj.lora_B.Llasa_adapter.weight | Trainable
Layer: vision_tower.vision_model.encoder.layers.1.self_attn.k_proj.lora_A.Llasa_adapter.weight | Trainable
Layer: vision_tower.vision_model.encoder.layers.1.self_attn.k_proj.lora_B.Llasa_adapter.weight | Trainable
Layer: vision_tower.vision_model.encoder.layers.1.self_attn.q_proj.lora_A.Llasa_adapter.weight | Trainable
Layer: vision_tower.vision_model.encoder.layers.1.self_attn.q_proj.lora_B.Llasa_adapter.weight | Trainable
Layer: vision_tower.vision_model.encoder.layers.2.self_attn.k_proj.lora_A.Llasa_adapter.weight | Trainable
Layer: vision_tower.vision_model.enco

In [12]:
from transformers import TrainingArguments, logging, Trainer
from model.llasa_processor import LlasaDataCollatorWithPadding

training_args = TrainingArguments(
    learning_rate = 2e-5,
    lr_scheduler_type = 'cosine',
    warmup_ratio = 0.1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    output_dir="tmp",

    logging_strategy='steps',
    logging_steps=200,
    logging_first_step=True,
    
    save_strategy='epoch',

    disable_tqdm=False,
    num_train_epochs=20,
    log_level='error',
    report_to='wandb',
    fp16=True,

    eval_steps=1,
    evaluation_strategy='epoch'

)

logging.set_verbosity_error()

trainer = Trainer(
    model=model, 
    args=training_args, 
    
    data_collator = LlasaDataCollatorWithPadding(processor=processor),
    train_dataset=train_dataset,
    eval_dataset = eval_dataset,
)

result = trainer.train()
print_summary(result)

/opt/conda/envs/llasa/lib/python3.11/site-packages/transformers/training_args.py:1539: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss


In [12]:
data = train_dataset[1]
#print(data)
#del data['seg_tokens']

In [16]:
generate_kwargs = {"max_new_tokens": 100, "do_sample": True, "top_p": 0.9}

output = model.generate(**data, **generate_kwargs)

with torch.no_grad():
    generated_text = processor.batch_decode(output, skip_special_tokens=False)

/opt/conda/envs/llasa/lib/python3.11/site-packages/transformers/generation/utils.py:1907: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [14]:
print(generated_text)

['<s> USER: <video>\nDescribe the scene and the behavior of the actor within it. Use segmentation to emphasize the actor and provide details about their actions. <seg>\nASSISTANT:ST booteton boot dom boot mentioness boot boot{ party blocks{ Boot h boot boot boot[ boot boot boot boot boot boot boot boot boot[ block boot block boot boot Boot boot boot boot boot boot boot boot boot{ boot boot, boot boot boot boot boot boot Boot bootBoot boot boot boot h boot boot boot Boot boot h boot boot h block[ boot boot boot boot bootess block boot block boot boot boot boot boot Boot boot boot Boot boot h h boot result boot boot boot boot boot']
